## Declaration and Preprocessing

In [1]:
import random
import math
import pandas as pd
import numpy as np
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
df = pd.read_excel('DATASET.xlsx')

In [ ]:
df.head()

In [5]:
#Anomolies 
clear_idx = 910
lst_to_clear = []
for i in range(9):
    lst_to_clear.append(clear_idx+i)
lst_to_clear
df.fillna(0, inplace=True)
df.drop(lst_to_clear, inplace = True)

In [6]:
df.columns = df.columns.str.replace('Unnamed: 0', 'Section_name')

In [ ]:

mul = 10
j = 0
for i in range(100):
    sec_name = df.iloc[j,0]
    df.iloc[j:j + mul, 0] = sec_name
    j += mul
test_frame = df.loc[df['year'] == 10]
df.head()

In [8]:
move = 9
def extract_xtrain(df, row_start, col_start):
    row_end = row_start + move
    return df.iloc[row_start:row_end, 1:10]

In [9]:
# Extract the training part of output dataset
def extract_ytrain(df, row_start, col_start):
    row_end = row_start + move
    return df.iloc[row_start:row_end, col_start]

In [10]:
# Extract the testing part of input dataset
def extract_xtest(df, row_start):
    row_end = row_start + move
    return df.iloc[row_end, 1:10].to_numpy()

In [11]:
# Extract the output part of ouput dataset
def extract_ytest(df, row_start, col_start):
    row_end = row_start + move
    return df.iloc[row_end, col_start]

In [12]:
# function to calcuate mean square error
def calcuate_lmse(predict_val, test_val):
    pred_diff = predict_val - test_val
    return (pred_diff)**2

In [13]:
# Get the XY Split
column_name = ["Para-"+str(i) for i in range(9,14)]
XY = []
def getXYsplit(df, row_start, row_end):
    XY = []
    XY.append(extract_xtrain(df, row_start, row_end))
    XY.append(extract_ytrain(df, row_start, row_end))
    XY.append(extract_xtest(df,row_start))
    XY.append(extract_ytest(df,row_start,row_end))
    return XY

In [14]:
from collections import defaultdict

In [15]:
# Create a dictionary for storing predicted and true values
pred_values = defaultdict(lambda : [])

In [16]:
from sklearn.metrics import mean_squared_error

In [17]:
rmse_combine = []

In [18]:
from sklearn.linear_model import BayesianRidge
model = BayesianRidge()

In [19]:
target_col = [10,11,12,13,14]
root_mean_square_error = []
total_count = 0
curr_section = 0
total_squared_error = 0
max_val = 1000
for j in target_col:
    local_squared_error = 0
    l_count = 0
    LXY = [0,0,0,0]
    for i in range(0, max_val, 10):
        LXY = getXYsplit(df,i,j)
        x_train, y_train,x_test,y_test = LXY[0],LXY[1],LXY[2],LXY[3]

        # fit the model and predict
        model.fit(x_train, y_train)
        ypred = model.predict(x_test.reshape(1,-1))[0]

        # store the predicted values
        pred_values[column_name[curr_section]].append(ypred)

        # store squared difference for RMSE

        # local_squared_error = calcuate_lmse(ypred, y_test) + local_squared_error
        total_squared_error = calcuate_lmse(ypred, y_test) + total_squared_error
        l_count += 1
        total_count += 1

    # root_mean_square_error.append(local_squared_error)
    curr_section += 1


In [ ]:
# Calculate  RMSE
total_rmse_by_n = total_squared_error/total_count
final_rmse = total_rmse_by_n**0.5
rmse_combine.append(["Bayesian Ridge",final_rmse])
print("The RMSE of Bayesian Ridge is -- " + str(final_rmse))

In [21]:
from sklearn.kernel_ridge import KernelRidge
model = KernelRidge(alpha=1.0)

In [22]:
target_col = [10,11,12,13,14]
root_mean_square_error = []
total_count = 0
curr_section = 0
total_squared_error = 0
max_val = 1000
for j in target_col:
    local_squared_error = 0
    l_count = 0
    LXY = [0,0,0,0]
    for i in range(0, max_val, 10):
        LXY = getXYsplit(df,i,j)
        x_train, y_train,x_test,y_test = LXY[0],LXY[1],LXY[2],LXY[3]

        # fit the model and predict
        model.fit(x_train, y_train)
        ypred = model.predict(x_test.reshape(1,-1))[0]

        # store the predicted values
        pred_values[column_name[curr_section]].append(ypred)

        # store squared difference for RMSE

        # local_squared_error = calcuate_lmse(ypred, y_test) + local_squared_error
        total_squared_error = calcuate_lmse(ypred, y_test) + total_squared_error
        l_count += 1
        total_count += 1

    # root_mean_square_error.append(local_squared_error)
    curr_section += 1


In [ ]:
# Calculate  RMSE
total_rmse_by_n = total_squared_error/total_count
final_rmse = total_rmse_by_n**0.5
rmse_combine.append(["Kernal Ridge",final_rmse])
print("The RMSE of Kernal Ridge  " + str(final_rmse))

In [24]:
## Define the Gradient Boosting
from sklearn.ensemble import GradientBoostingRegressor
model = GradientBoostingRegressor(random_state=42)

In [25]:
target_col = [10,11,12,13,14]
root_mean_square_error = []
total_count = 0
curr_section = 0
total_squared_error = 0
max_val = 1000
for j in target_col:
    local_squared_error = 0
    l_count = 0
    LXY = [0,0,0,0]
    for i in range(0, max_val, 10):
        LXY = getXYsplit(df,i,j)
        x_train, y_train,x_test,y_test = LXY[0],LXY[1],LXY[2],LXY[3]

        # fit the model and predict
        model.fit(x_train, y_train)
        ypred = model.predict(x_test.reshape(1,-1))[0]

        # store the predicted values
        pred_values[column_name[curr_section]].append(ypred)

        # store squared difference for RMSE

        # local_squared_error = calcuate_lmse(ypred, y_test) + local_squared_error
        total_squared_error = calcuate_lmse(ypred, y_test) + total_squared_error
        l_count += 1
        total_count += 1

    # root_mean_square_error.append(local_squared_error)
    curr_section += 1


In [ ]:
# Calculate  RMSE
total_rmse_by_n = total_squared_error/total_count
final_rmse = total_rmse_by_n**0.5
rmse_combine.append(["Gradient Boosting",final_rmse])
print("The RMSE of Gradient Boosting  " + str(final_rmse))

# Plot

In [ ]:
# Get random rows for plotting
plot_row = 50

random_rows = []
for i in range(plot_row):
    random_rows.append(random.randint(0,90))


In [ ]:
figure(figsize=(13, 7), dpi=130)
true_value = predicted_df.iloc[random_rows, 0]
predicted_value = test_frame.iloc[random_rows,10]
x_value = []
for i in range(len(true_value)):
    x_value.append(i+1)
plt.plot(x_value,predicted_value, linestyle = 'dotted', label = 'predicted')
plt.plot(x_value,true_value, linestyle = 'dotted', label = 'orginial')
plt.xlabel('Number of Values')
plt.ylabel('Values')
plt.title('True vs Predicted Values of Para-9')
plt.legend(loc = 'upper right')

In [ ]:
figure(figsize=(13, 7), dpi=130)
true_value = predicted_df.iloc[random_rows, 0]
predicted_value = test_frame.iloc[random_rows,11]
x_value = []
for i in range(len(true_value)):
    x_value.append(i+1)
plt.plot(x_value,predicted_value, linestyle = 'dotted', label = 'predicted')
plt.plot(x_value,true_value, linestyle = 'dotted', label = 'orginial')
plt.xlabel('Number of Values')
plt.ylabel('Values')
plt.title('True vs Predicted Values of Para-10')
plt.legend(loc = 'upper right')

In [ ]:
figure(figsize=(13, 7), dpi=130)
true_value = predicted_df.iloc[random_rows, 0]
predicted_value = test_frame.iloc[random_rows,12]
x_value = []
for i in range(len(true_value)):
    x_value.append(i+1)
plt.plot(x_value,predicted_value, linestyle = 'dotted', label = 'predicted')
plt.plot(x_value,true_value, linestyle = 'dotted', label = 'orginial')
plt.xlabel('Number of Values')
plt.ylabel('Values')
plt.title('True vs Predicted Values of Para-11')
plt.legend(loc = 'upper right')

In [ ]:
figure(figsize=(13, 7), dpi=130)
true_value = predicted_df.iloc[random_rows, 0]
predicted_value = test_frame.iloc[random_rows,13]
x_value = []
for i in range(len(true_value)):
    x_value.append(i+1)
plt.plot(x_value,predicted_value, linestyle = 'dotted', label = 'predicted')
plt.plot(x_value,true_value, linestyle = 'dotted', label = 'orginial')
plt.xlabel('Number of Values')
plt.ylabel('Values')
plt.title('True vs Predicted Values of Para-12')
plt.legend(loc = 'upper right')

In [ ]:
figure(figsize=(13, 7), dpi=130)
true_value = predicted_df.iloc[random_rows, 0]
predicted_value = test_frame.iloc[random_rows,14]
x_value = []
for i in range(len(true_value)):
    x_value.append(i+1)
plt.plot(x_value,predicted_value, linestyle = 'dotted', label = 'predicted')
plt.plot(x_value,true_value, linestyle = 'dotted', label = 'orginial')
plt.xlabel('Number of Values')
plt.ylabel('Values')
plt.title('True vs Predicted Values of Para-13')
plt.legend(loc = 'upper right')

In [ ]:
rmse_combine